In [12]:

import datetime
import pandas as pd
import re
from datetime import timedelta








### Loading Data
We start by loading the two datasets that we imported from your fleet tacking app: the drivers list and the trips dataset. We use pandas, to load our data.



In [13]:
driver_data = pd.read_csv("Data/driver_data .csv")
trip_data = pd.read_csv("Data/trip_data (3).csv")


### Data Preparation
From the 'trip_data' dataframe, we select the columns that are relevant for our analysis and create a new dataframe, 'Trips'. We rename the 'id' column to 'trip_id' to avoid confusion with the 'id' in the driver_data dataframe.
(please explore your data to make you grab all the relevent features) 

In [14]:
Trips = trip_data[['id', 'start', 'stop', 'stopDuration', 'driver', 'averageSpeed', 'distance', 'drivingDuration']].copy()
Trips = Trips.rename(columns={'id': 'trip_id'})


In [15]:
# From the 'driver' column in the 'Trips' dataframe, we extract the driver IDs using regular expressions and store them in a new column, 'id'. 
pattern = r"'id': '(\w+)'"
Trips['id'] = Trips['driver'].str.extract(pattern, expand=False)

# We then drop the 'driver' column as it is no longer needed.
Trips = Trips.drop(columns=['driver'])



In [16]:
#let us define a fonction to convert string format to timedelta object.
def parse_duration(duration_str):
    days, hours, minutes, seconds = 0, 0, 0, 0
    match = re.match(r'(\d+)\.(\d+):(\d+):(\d+)', duration_str)
    if match:
        days, hours, minutes, seconds = map(int, match.groups())
    else:
        match = re.match(r'(\d+):(\d+):(\d+)', duration_str)
        if match:
            hours, minutes, seconds = map(int, match.groups())
    return timedelta(days=days, hours=hours, minutes=minutes, seconds=seconds)
# We convert the 'stopDuration' and 'drivingDuration' columns from string format to timedelta objects using our function,
Trips['stopDuration'] = Trips['stopDuration'].apply(parse_duration)
Trips['drivingDuration'] = Trips['drivingDuration'].apply(parse_duration)


In [17]:
# We convert 'start' and 'stop' columns into Timestamp objects 
Trips["start"] = pd.to_datetime(Trips["start"])
Trips["stop"] = pd.to_datetime(Trips["stop"])
# and transform the 'drivingDuration' column into the number of hours

Trips["hours driving"] = Trips["drivingDuration"].dt.total_seconds()/3600 # convert to hours
#We also extract the date and time components from the 'start' and 'stop' columns into separate columns.
Trips['start_date'] = Trips['start'].dt.date
Trips['start_time'] = Trips['start'].dt.time
Trips['stop_date'] = Trips['stop'].dt.date
Trips['stop_time'] = Trips['stop'].dt.time


### Data Aggregation
We summarize the trip data on a daily basis for each driver using the groupby function. The agg function is then used to perform several aggregation operations such as count, max, min, sum, and mean on the relevant columns.

Then, we merge this summarized data with the driver_data dataframe to include driver names in the final summary.


In [18]:
summary_trips = Trips.groupby(["id",Trips["start_date"]]).agg({'trip_id': "count",'stop_time': 'max', 'start_time': 'min','stop': 'max', 'start': 'min', "distance" : "sum" ,"averageSpeed": "mean", 'trip_id': "count",'hours driving':"sum","drivingDuration":"mean","stopDuration":"mean"}).reset_index()
summary_trips= summary_trips.merge(driver_data[['id', 'Name']], on='id')

#now our final DF is ready for Analysis
summary_trips

,id,start_date,trip_id,stop_time,start_time,stop,start,distance,averageSpeed,hours driving,drivingDuration,stopDuration,Name
0,b116,2022-06-20,19,21:10:29,10:09:33,2022-06-20 21:10:29+00:00,2022-06-20 10:09:33+00:00,138.112091,21.606314,3.584722,0 days 00:11:19.210526315,0 days 01:05:28.947368421,Alvin Fuller
1,b116,2022-06-21,19,20:52:12,10:28:49,2022-06-21 20:52:12+00:00,2022-06-21 10:28:49+00:00,124.697151,18.156447,2.864722,0 days 00:09:02.789473684,0 days 00:33:22.842105263,Alvin Fuller
2,b116,2022-06-22,21,20:10:48,10:35:41,2022-06-22 20:10:48+00:00,2022-06-22 10:35:41+00:00,127.166149,19.187518,3.218333,0 days 00:09:11.714285714,0 days 00:20:38.285714285,Alvin Fuller
3,b116,2022-06-23,16,19:32:00,10:36:30,2022-06-23 19:32:00+00:00,2022-06-23 10:36:30+00:00,132.406526,22.406037,3.633056,0 days 00:13:37.437500,0 days 00:28:38.625000,Alvin Fuller
4,b116,2022-06-27,17,18:48:25,10:18:05,2022-06-27 18:48:25+00:00,2022-06-27 10:18:05+00:00,128.245914,22.028013,2.605833,0 days 00:09:11.823529411,0 days 01:15:06.764705882,Alvin Fuller
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216,b90,2022-08-01,17,20:37:31,09:48:21,2022-08-01 20:37:31+00:00,2022-08-01 09:48:21+00:00,488.337406,15.870442,7.748333,0 days 00:27:20.823529411,0 days 00:12:27.117647058,Robert Taylor
1217,b90,2022-08-02,10,20:41:35,09:44:46,2022-08-02 20:41:35+00:00,2022-08-02 09:44:46+00:00,366.736505,26.891061,5.074722,0 days 00:30:26.900000,0 days 00:37:35.800000,Robert Taylor
1218,b90,2022-08-03,8,19:05:42,09:54:43.063000,2022-08-03 19:05:42+00:00,2022-08-03 09:54:43.063000+00:00,579.515747,39.722087,6.928611,0 days 00:51:57.875000,0 days 00:18:12.375000,Robert Taylor
1219,b90,2022-08-04,13,18:38:31,09:46:51,2022-08-04 18:38:31+00:00,2022-08-04 09:46:51+00:00,522.136799,23.707006,6.686389,0 days 00:30:51.615384615,0 days 00:14:44.153846153,Robert Taylor
